In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from mpl_toolkits.mplot3d import Axes3D
import netCDF4
from netCDF4 import Dataset
import numpy
import os
import pandas
from PIL import Image
import random
import readline
import scipy
from scipy import fft
from scipy import linalg
from scipy import stats
from scipy.stats import poisson, ttest_ind
import seaborn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import normalize
from statsmodels.tsa.ar_model import AutoReg
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Github/RCP_Figs/Analysis_Figs_V6/'
Compo_Diri = '/glade/u/home/whimkao//ExtraTrack/Composite_Files_V5/'

In [3]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [4]:
# Open Each File
def Files_Open(Model, Diri):
    Data_DF = Open_File(Diri+Model+'_Data_All_Output_V6.csv')
    ET_DF = Open_File(Diri+Model+'_ET_All_Output_V6.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V6.csv')
    Time = []
    Birth_Time = []
    Start_Time = []
    End_Time = []
    Death_Time = []
    Peak_Time = []
# Edit Time Format
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Birth_Time.append(Datetime(ET_DF["Genesis Time"][j]))
        Start_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        End_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Death_Time.append(Datetime(ET_DF["Dissipate Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["Genesis Time"] = Birth_Time
    ET_DF["ET Begin Time"] = Start_Time
    ET_DF["ET Complete Time"] = End_Time
    ET_DF["Dissipate Time"] = Death_Time
    ET_DF["Peak Time"] = Peak_Time
    return (Data_DF, ET_DF, Codes_DF)

In [5]:
def Datetime(Time):
    try:
        New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    except:
        New_Time = numpy.nan
    return (New_Time)

In [6]:
Control_Data_All, Control_ET_All, Control_Codes = Files_Open("Control", Diri)
RCP45_Data_All, RCP45_ET_All, RCP45_Codes = Files_Open("RCP45", Diri)
RCP85_Data_All, RCP85_ET_All, RCP85_Codes = Files_Open("RCP85", Diri)

In [7]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [8]:
# Create Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)

In [9]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [10]:
Control_ET_All[Control_ET_All["Genesis SLP"] < 985][["Code", "Name", "Genesis SLP", "Genesis Lon", "Genesis Lat", "Genesis Time"]]

,Code,Name,Genesis SLP,Genesis Lon,Genesis Lat,Genesis Time
12,TC190109,Shannon,969.93,-45.76,43.50,1901-12-10 06:00:00
23,TC190302,Katrina,981.27,-64.30,38.05,1903-05-10 18:00:00
36,TC190501,Grace,977.41,-30.28,33.82,1905-01-02 18:00:00
46,TC190601,Bryan,978.57,-42.44,36.52,1906-01-18 12:00:00
57,TC190701,Tess,981.62,-50.72,37.95,1907-01-07 06:00:00
95,TC191202,Travis,951.00,-47.73,43.15,1912-02-15 00:00:00
197,TC192302,Ryan,980.07,-52.77,39.87,1923-04-02 12:00:00
309,TC193604,Quinn,974.21,-26.35,39.11,1936-03-21 00:00:00
343,TC193911,Nabil,972.08,-55.53,39.64,1939-11-18 06:00:00
359,TC194102,Romeo,978.41,-54.52,36.70,1941-03-04 06:00:00


In [11]:
RCP45_ET_All[RCP45_ET_All["Genesis SLP"] < 985][["Code", "Name", "Genesis SLP", "Genesis Lon", "Genesis Lat", "Genesis Time"]]

,Code,Name,Genesis SLP,Genesis Lon,Genesis Lat,Genesis Time
0,TC200001,Alex,977.46,-46.32,40.22,2000-01-12 18:00:00
1,TC200002,Beatrice,977.86,-55.37,40.08,2000-01-29 12:00:00
2,TC200003,Colin,983.47,-54.74,32.41,2000-02-25 00:00:00
51,TC200707,Aidan,983.55,-63.47,36.05,2007-12-17 18:00:00
76,TC201110,Ian,982.67,-13.70,38.99,2011-12-07 18:00:00
83,TC201301,Sofiee,966.63,-47.24,42.86,2013-01-19 06:00:00
98,TC201504,Clark,966.99,-58.76,37.81,2015-03-12 00:00:00
121,TC201908,Zack,942.64,-47.44,44.80,2019-12-10 18:00:00
123,TC202002,Beatrice,984.24,-61.32,38.37,2020-02-19 06:00:00
164,TC202603,Gabriel,973.45,-49.38,40.03,2026-12-27 00:00:00


In [12]:
RCP85_ET_All[RCP85_ET_All["Genesis SLP"] < 985][["Code", "Name", "Genesis SLP", "Genesis Lon", "Genesis Lat"]]

,Code,Name,Genesis SLP,Genesis Lon,Genesis Lat
1,TC210003,Colin,953.85,-47.15,41.53
5,TC210007,Harrison,964.17,-71.18,35.13
15,TC210201,Audrey,973.28,-61.53,37.64
32,TC210604,Noelle,951.80,-63.02,40.45
58,TC211301,Christy,972.04,-73.38,34.00
80,TC211604,Sofiee,974.11,-49.94,37.64
82,TC211606,Yan,979.74,-40.60,39.33
84,TC211706,Ivanka,960.98,-41.68,37.51
85,TC211707,Jack,973.77,-56.33,37.40
124,TC212504,Nicole,977.85,-61.70,40.19


In [13]:
# Create Function to Open Storm Composite Files
def Composite_File(File):
    Diri = '/glade/work/whimkao//ExtraTrack/Composite_Files_V5/'
    Compo_File = xr.open_dataset(Diri + File)
    return (Compo_File)

In [14]:
# Open Storm Composite Files
Control_A_Compo_nc = Composite_File('composite_dtime900.control.002.nc')
Control_B_Compo_nc = Composite_File('composite_dtime900.control.003.nc')
Control_C_Compo_nc = Composite_File('composite_dtime900.control.004.nc')
Control_A_Compo_nc

FileNotFoundError: [Errno 2] No such file or directory: '/glade/work/whimkao/ExtraTrack/ExtraTrack_Data/Composite_Files_V5/composite_dtime900.control.002.nc'